In [1]:
"""Лабораторная № 1
Задание 1
Решение уравнений с одной переменной"""

import numpy as np
import time

def print_dichotomy(dichotomy):
    def wrapper(a,b,n, eps):
        start_time = time.time()
        res = list(dichotomy(a,b,n, eps))
        func = ', '.join(map(lambda x: f'{f(x):.4f}', res))
        print('Затраченное время по версии модуля time: %s секунды' % (time.time() - start_time))
        print('Корни по методу дихотомии находятся в точках:')
        print(', '.join(map(lambda x: f'{x:.4f}', res)))
        print('Значения функции составляют:\n', func)
        print('Количество значений функции: ', len(func.split() ) )
        print('Количество итераций: ', counter)
        print('Заданная точность по аргументу: ', eps)
    return wrapper

In [2]:
def f(x):
    return 1.2-np.log(x)-4*np.cos(2*x)

In [3]:
@print_dichotomy
def dichotomy(a, b, n, eps):  
    """
    Функция отделения и уточнения корня.
    Передаются параметры: отрезок от a до b делим на n частей, погрешность eps
    """

    global counter
    assert a != 0, 'a равно 0'
    assert b != 0, 'b равно 0'

    # сначала отделим корни
    grid = np.linspace(a, b, n)

    # далее уточним корни
    counter = 0
    for x, y in zip(grid, grid[1:]):
        # если на отрезке нет корня, смотрим следующий
        if f(x) * f(y) > 0:  
            continue
        root = None
        # пока отрезок больше заданной погрешности, выполняем нижестоящие операции:
        while (abs(f(y) - f(x))) > eps:
            # получаем середину отрезка
            mid = (y + x) / 2
            # пополним счетчик итераций на единицу:
            counter += 1
            # если функция в середине отрезка равну нулю или меньше погрешности:
            if f(mid) == 0 or f(mid) < eps:
                # корень равень серединному значению:
                root = mid  
                break
            # иначе если произведение функции в середине отрезка на функцию в т. а <0:
            elif (f(mid) * f(x)) < 0:
                # серединой становится точка b:
                y = mid  
            else:
                # в другом случае — точка а:
                x = mid  
        if root:
            yield root

In [4]:
dichotomy(0.0001, 50, 10000, 0.000001)

Затраченное время по версии модуля time: 0.13434815406799316 секунды
Корни по методу дихотомии находятся в точках:
0.0639, 0.5527, 2.3978, 3.9480, 5.4381, 7.1633, 8.5206, 10.3536, 11.6237, 13.5289, 14.7340, 16.6992, 17.8493, 19.8645, 20.9665, 23.0286, 24.0900, 26.1889, 27.2121, 29.3455, 30.3378, 32.5045, 33.4659, 35.6598, 36.5924, 38.8152, 39.7215, 41.9695, 42.8518, 45.1220, 45.9821, 48.2755, 49.1124
Значения функции составляют:
 -0.0163, -0.0024, -0.0073, -0.0053, -0.0173, -0.0157, -0.0013, -0.0055, -0.0158, -0.0170, -0.0175, -0.0145, -0.0159, -0.0161, -0.0013, -0.0046, -0.0125, -0.0035, -0.0007, -0.0156, -0.0018, -0.0017, -0.0109, -0.0050, -0.0029, -0.0023, -0.0054, -0.0005, -0.0110, -0.0061, -0.0125, -0.0025, -0.0107
Количество значений функции:  33
Количество итераций:  55
Заданная точность по аргументу:  1e-06


In [8]:
"""Лабораторная № 1
Задание 2
Решение систем линейных уравнений"""

# import numpy as np

def print_gaussFunc(make_identity):
    def gaussFunc(matrix):
        """
        Функция меняет матрицу через побочные эффекты.
        Если нам потребуется сохранить прежнюю матрицу, можно скопировать её методом np.copy
        Это прямой ход метода Гаусса 
        """

        for nrow, row in enumerate(matrix):
            # nrow равен номеру строки
            # row содержит саму строку матрицы
            # перед тем, как делить на диагональный элемент, сначала просмотрим все строки, начиная с текущей, вниз:
            # выберем строку с максимальным значением в текущей колонке и переставим с текущей. После чего продолжим.
            # Это надо сделать перед делением на диагональный элемент, т.к.
            # в ходе вычислений на диагонали может оказаться ноль.
            # Цикл поиска максимума в столбце:

            max_elem = abs(row[nrow])
            max_row = nrow
            # хранит размер матрицы matrix n (len(matrix[:, 0]) - количество строк в столбце 0
            for i in range(nrow + 1, len(matrix)): 
                # matrix[i][nrow] проходит по элементам под главной диагональю:
                if abs(matrix[i][nrow]) > abs(max_elem):  
                    max_elem = matrix[i][nrow]
                    # фиксируем строку с макс. элементом:
                    max_row = i  
            if max_row != nrow:
                # меняем текущую строку со строкой, в которой максимум (реализация swap()):
                matrix[nrow], matrix[max_row] = matrix[max_row], np.copy(matrix[nrow])
                # также можно использовать такую реализацию перестановки строк с индексным массивом:
                # matrix[[nrow, max_row]] = matrix[[max_row, nrow]]
            row = matrix[nrow]
            # диагональный элемент
            divider = row[nrow]  
            if abs(divider) < 1e-10:
                # почти ноль на диагонали. Продолжать не имеет смысла, результат счёта неустойчив
                raise ValueError(f"Матрица несовместна. Максимальный элемент в столбце {nrow}: {divider:.3g}")
            # делим на диагональный элемент:
            row /= divider
            # теперь вычтем приведённую строку из всех нижележащих строк:
            for lower_row in matrix[nrow+1:]:
                # элемент строки в колонке nrow (другая запись matrix[i][nrow])
                factor = lower_row[nrow]  
                # вычитаем, чтобы получить ноль в колонке nrow
                lower_row -= factor * row  
        # все строки матрицы изменились, в принципе, можно и не возвращать
        print('Матрица, полученная методом Гаусса (промежуточная матрица):\n', matrix)
        make_identity(np.copy(matrix))
    return gaussFunc


In [9]:
@print_gaussFunc
def make_identity(gauss_matrix):
    """Функция берёт матрицу, полученную методом Гаусса, и доводит её до единичной.
    Для этого строки перебираются снизу вверх и вычитаются из вышележащих строк,
    чтобы обнулить соответствующие колонки"""
    # перебор строк в обратном порядке (старт: len(matrix)-1; стоп: 0; шаг: -1, т.е. обратный ход):
    for nrow in range(len(gauss_matrix)-1, 0, -1):
        # строки начиная с последней:
        row = gauss_matrix[nrow]
        # идем по строкам (upper_row) сверху: 
        for upper_row in gauss_matrix[:nrow]:
            # элемент (factor) строки в колонке nrow (элементы над главной диагональю (остались необнуленными)):
            factor = upper_row[nrow]  
            upper_row -= factor * row
    print('Корни системы уравнений:\n', gauss_matrix[:,3])
    roots = gauss_matrix[:,3]
    print('Невязка (ожидаются нули):\n', np.matmul(gauss_matrix[:,:3], roots.T) - gauss_matrix[:,3])
    return gauss_matrix[:,3]

In [10]:
gauss_matrix = np.array([[3.8, 6.7, -1.2, 5.2],
                   [6.4, 1.3, -2.7, 3.8],
                   [2.4, -4.5, 3.5, -0.6]])
make_identity(gauss_matrix)

Матрица, полученная методом Гаусса (промежуточная матрица):
 [[ 1.          0.203125   -0.421875    0.59375   ]
 [ 0.          1.          0.06800211  0.49657354]
 [ 0.          0.          1.          0.09309401]]
Корни системы уравнений:
 [0.53344344 0.49024295 0.09309401]
Невязка (ожидаются нули):
 [0. 0. 0.]


In [11]:
"""
Лабораторная № 1
Задание 3
Расчет определителя матрицы 
"""

def determinant(array):
    return array[0][0] * array[1][1] - array[1][0] * array[0][1]



In [16]:
def division(array):
    if len(array[0]) > 2:
        result = 0
        for i in range(len(array[0])):
            new_arr = []
            for j in range(len(array[0])):
                if j != i:
                    new_arr.append([array[j][k] for k in range(1, len(array[0]))])
            print('Промежуточная матрица №:', i+1, '\n', np.matrix(new_arr))
            result += division(new_arr) * array[i][0] * (-1 + 2 * ((i + 1) % 2))
        print('Детерминант матрицы:', result)
    else:
        return determinant(array)

In [17]:
matrix = np.array([[3.8, 6.7, -1.2], 
                   [6.4, 1.3, -2.7], 
                   [2.4, -4.5, 3.5]])
division(matrix)

Промежуточная матрица №: 1 
 [[ 1.3 -2.7]
 [-4.5  3.5]]
Промежуточная матрица №: 2 
 [[ 6.7 -1.2]
 [-4.5  3.5]]
Промежуточная матрица №: 3 
 [[ 6.7 -1.2]
 [ 1.3 -2.7]]
Детерминант матрицы: -184.072


In [19]:
"""
Лабораторная № 1
Задание 4
Написать программу вычисления обратной матрицы одним из методов
"""

def print_inverse_matrix(inverse_matrix):
    def wrapper(matrix_origin):
        print('Обратная матрица, полученная методом Гаусса:\n', '\n', inverse_matrix(np.copy(matrix)))
        print()
        print('Невязка:\n', np.max(np.abs(np.matmul(matrix, inverse_matrix(np.copy(matrix))) - np.eye(3))))
    return wrapper

In [20]:
@print_inverse_matrix
def inverse_matrix(matrix_origin):
    """
    Функция получает на вход матрицу, затем добавляет к ней единичную матрицу,
    проводит элементарные преобразования по строкам с первоначальной, добиваясь получения слева единичной матрицы.
    В этом случае справа окажется матрица, которая является обратной к заданной первоначально
    """
    # Склеиваем 2 матрицы: слева - первоначальная, справа - единичная
    n = matrix_origin.shape[0]
    matrix = np.hstack((matrix_origin, np.eye(n)))

    for nrow, row in enumerate(matrix):
        # nrow равен номеру строки
        # row содержит саму строку матрицы
        divider = row[nrow]  # диагональный элемент
        # делим на диагональный элемент:
        row /= divider
        # теперь вычитаем приведённую строку из всех нижележащих строк:
        for lower_row in matrix[nrow + 1:]:
            factor = lower_row[nrow]  # элемент строки в колонке nrow
            lower_row -= factor * row  # вычитаем, чтобы получить ноль в колонке nrow

    # обратный ход
    for nrow in range(len(matrix)-1, 0, -1):
        row = matrix[nrow]
        for upper_row in matrix[:nrow]:
            factor = upper_row[nrow]
            # Вычитаем целую строку:
            upper_row -= factor * row
    return matrix[:, n:].copy()

In [21]:
my_matrix = np.array([[3.8, 6.7, -1.2], 
                   [6.4, 1.3, -2.7], 
                   [2.4, -4.5, 3.5]])

In [22]:
inverse_matrix(my_matrix)

Обратная матрица, полученная методом Гаусса:
 
 [[ 0.04128819  0.09805945  0.08980182]
 [ 0.15689513 -0.08790039 -0.01401625]
 [ 0.1734104  -0.18025555  0.206115  ]]

Невязка:
 1.3547119009181044e-16
